In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam


In [17]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")


In [ ]:
# Paths to training and validation datasets
train_data_path = 'C:/Users/ambik/Desktop/AI Project/ProjectFiles/data2/train'
val_data_path = 'C:/Users/ambik/Desktop/AI Project/ProjectFiles/data2/validation'

# Data generators with augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation data
val_datagen = ImageDataGenerator(rescale=1.0/255)

# Generate training and validation data
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 7236 images belonging to 2 classes.
Found 1818 images belonging to 2 classes.


In [14]:
# Load the VGG19 model without the top (classification) layer and with pre-trained ImageNet weights
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of VGG19 to prevent training them
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG19
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [18]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Epoch 1/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 2228s 10s/step - accuracy: 0.8444 - loss: 0.3416 - val_accuracy: 0.8647 - val_loss: 0.2963
Epoch 2/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 2156s 10s/step - accuracy: 0.8480 - loss: 0.3410 - val_accuracy: 0.8553 - val_loss: 0.3175
Epoch 3/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 2192s 10s/step - accuracy: 0.8461 - loss: 0.3316 - val_accuracy: 0.8768 - val_loss: 0.2827
Epoch 4/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 2204s 10s/step - accuracy: 0.8519 - loss: 0.3309 - val_accuracy: 0.8608 - val_loss: 0.2870
Epoch 5/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 2118s 9s/step - accuracy: 0.8425 - loss: 0.3313 - val_accuracy: 0.8707 - val_loss: 0.2926
Epoch 6/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 2054s 9s/step - accuracy: 0.8477 - loss: 0.3295 - val_accuracy: 0.8663 - val_loss: 0.2875
Epoch 7/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 2084s 9s/step - accuracy: 0.8585 - loss: 0.3137 - val_accuracy: 0.8702 - val_loss: 0.2827
Epoch 8/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 2218s 10s/step - accuracy: 0.8661 - loss: 0.30

In [19]:
# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

57/57 ━━━━━━━━━━━━━━━━━━━━ 424s 7s/step - accuracy: 0.8689 - loss: 0.2881
Validation Loss: 0.30517974495887756
Validation Accuracy: 0.8586358428001404


In [20]:
model.save("parkinsons_detection_model.h5")


In [21]:
model.save("parkinsons_detection_model.keras")
